Hi, I'm ÉléonORe, and I'm struggling to organize a series of events...

Each event requires a dedicated room.

Some events have overlapping participants, so I can't schedule them in the same room at the same time.

How many rooms do I need? How can I assign each event to a room so that no two overlapping events are scheduled in the same one?

Ideally, I want to minimize the total number of rooms used.

Can you help me solve this problem?

## Solution modeled with binary variables

**Goal**

Minimize number of rooms to be used during the whole series of events.

$\min \sum_{r=0}^{m} y_r$

**Decisions**:

$y_r\in\{0,1\}$ to use room r

$x_{er}\in\{0,1\}$ to schedule event $e$ into room $r$


**Data:**

Set of events $E=\{1..n\}$

Set of rooms $R = \{1..n\}$ (in the worst-case scenario each event is assigned to its own room)

Conflict paris $C = \{(i,j)|i,j\in E\}$

**Constraints**

| Description    | Expression |
| -------- | ------- |
| One room for each event e  | $\forall e \in E: \sum_r x_{er} = 1$    |
| No conflict for each conflict pair (i,j) | $\forall (i, j) \in C, r \in R: x_{ir} + x_{jr} \leq 1$ |
| Room is used only if an event is assigned to it | $\forall r \in R: \sum_e x_{er} \leq y_r$ |

In [36]:
def get_instance(input_filename):
    with open(input_filename, "r") as file:
        data = file.read()

    # Split the data into lines
    lines = data.strip().split('\n')
    l = 0

    # Skip comment lines
    while lines[l].startswith('#') or lines[l] == '':
        l += 1
    
    # get metadata parameters (always first line)
    metadata = map(int, lines[l].split())
    data = []
    l += 1

    # Process each line
    while l < len(lines):
        data.append(list(map(int, lines[l].split())))
        l += 1

    return *metadata, data

In [55]:
num_events, num_conflicts, conflicts = get_instance("data/conflicting_events.txt")

#num_rooms = num_events # worst-case scenario
num_rooms = 15
# Print the extracted data
print(f"Number of events: {num_events}")
print(f"Number of rooms: {num_rooms}")
print(f"Number of conflicts: {num_conflicts}")
# print("Conflicts:")
# for conflict in conflicts:
#     print(conflict)

Number of events: 100
Number of rooms: 15
Number of conflicts: 2487


In [56]:
import pandas as pd
from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()


In [57]:
y = [model.NewBoolVar(name=f'is_room_{r}_active') for r in range(num_rooms)]
x = [[model.NewBoolVar(name=f'event_{e}_in_room_{r}') for r in range(num_rooms)] for e in range(num_events)]

In [58]:
for e in range(num_events):
    model.Add(sum(x[e]) == 1)

In [ ]:
for e1, e2 in conflicts:
    e1 -= 1 #parsing to 0-indexed array
    e2 -= 1
    for r in range(num_rooms):
        model.Add(x[e1][r] + x[e2][r] <= 1)

In [60]:
# for r, x_room in enumerate(zip(*x)):
#     model.Add(sum(x_room) <= y[r])

for r in range(num_rooms):
    for e in range(num_events):
        model.Add(x[e][r] <= y[r])

In [63]:
# Specify the type of problem. In this case, we want to minimize the objective function
solver.parameters.num_search_workers = 8
solver.parameters.max_time_in_seconds = 120
model.Minimize(sum(y))

In [64]:
# Call the solver method to find the optimal solution
callback = cp_model.ObjectiveSolutionPrinter()
or_status = solver.SolveWithSolutionCallback(model, callback)
status = solver.StatusName(or_status)

if status in ["OPTIMAL", "FEASIBLE"]:
    print(f'Solution: Total rooms used = {solver.ObjectiveValue()}')
else:
    print('A solution could not be found, check the problem specification')

Solution 0, time = 0.66 s, objective = 15
Solution: Total rooms used = 15.0
